In [11]:
# dependencies installation
%pip install pymupdf
%pip install frontend
%pip install pillow


Note: you may need to restart the kernel to use updated packages.



In [12]:
#Size of scale declaration
scaleX=108
scaleY=108


#imprting  
from PIL import Image, ImageDraw
from io import BytesIO
import fitz

#read from csv file and open it
pdf_dir = "./Data Extraction/Train Files/1040_file_2.pdf" 
doc = fitz.open(pdf_dir)
page = doc[0]

blocks = page.get_text("dict")['blocks']
text_blocks = [block for block in blocks if block['type'] == 0]
color = 'red'
width = 2
TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI / scaleX, TARGET_DPI / scaleY)
pix_map = doc[0].get_pixmap(matrix=mat)
image = Image.open(BytesIO(pix_map.tobytes()))
draw = ImageDraw.Draw(image)
''' 
Below are the coordinates of bbox which are extracted and  modified with respect to DE-TRAIN.csv file

'''

datas = [
    {"title": "Basic Information___First Name", "coordinate": [100, 325, 650, 365]},
    {"title": "Basic Information___Last Name", "coordinate": [675, 325, 1300, 365]},
    {"title": "Income___One_A", "coordinate": [1400, 1015, 1600, 1049]},
    {"title": "Income___One_B", "coordinate": [1400, 1049, 1600, 1083]},
    {"title": "Income___One_C", "coordinate": [1400, 1083, 1600, 1117]},
    {"title": "Income___One_D", "coordinate": [1400, 1112, 1600, 1150]},
    {"title": "Income___One_E", "coordinate": [1400, 1150, 1600, 1184]},
    {"title": "Income___One_F", "coordinate": [1400, 1180, 1600, 1219]},
    {"title": "Income___One_G", "coordinate": [1400, 1217, 1600, 1250]},
    {"title": "Income___One_H", "coordinate": [1400, 1250, 1600, 1284]},
    {"title": "Income___One_Z", "coordinate": [1400, 1284, 1600, 1352]},
    {"title": "Income___Two_B", "coordinate": [1400, 1352, 1600, 1384]},
    {"title": "Income___Three_B", "coordinate": [1400, 1384, 1600, 1417]},
    {"title": "Income___Four_B", "coordinate": [1400, 1417, 1600, 1450]},
    {"title": "Income___Five_B", "coordinate": [1400, 1450, 1600, 1484]},
    {"title": "Income___Six_B", "coordinate": [1400, 1484, 1600, 1515]},
    {"title": "Income___Seven",   "coordinate": [1400, 1515, 1600, 1585]},
    {"title": "Income___Eight",   "coordinate": [1400, 1585, 1600, 1618]},
    {"title": "Income___Nine",   "coordinate": [1400, 1618, 1600, 1650]},
    {"title": "Income___Ten",  "coordinate": [1400, 1650, 1600, 1685]},
    {"title": "Income___Eleven",  "coordinate": [1400, 1680, 1600, 1719]},
    {"title": "Income___Twelve",  "coordinate": [1400, 1715, 1600, 1752]},
    {"title": "Income___Thirteen",  "coordinate": [1400, 1748, 1600, 1785]},
    {"title": "Income___Fourteen",  "coordinate": [1400, 1780, 1600, 1819]},
    {"title": "Income___Fifteen",  "coordinate": [1400, 1814, 1600, 1854]},
]

'''
Drawing the outline box from the given co ordinates mentioned in the variable datas.
'''

for data in datas:
    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                
                x0, y0, x1, y1 =data["coordinate"]
                # x0, y0, x1, y1 =111,329,639,361
                if x0 > x1:
                    x0, x1 = x1, x0
                if y0 > y1:
                    y0, y1 = y1, y0
            draw.rectangle((x0, y0, x1, y1), fill=None, outline='green', width=width)
            # print(x0,y0,x1,y1)

image.show()


In [13]:

import pandas as pd
df = pd.DataFrame(columns=["file_name",'label', 'value', 'x0', 'y0', 'x2', 'y2'])

In [14]:
'''
This function is used to extract the exact co ordinates of the targeted text . 
It takes inputs of path of pdf and the targeted text 

'''
def extract_text_from_coordinates(pdf_dir, target_text):
    doc = fitz.open(pdf_dir)
    page = doc[0]
    
    target_x0, target_y0, target_x1, target_y1 = 0, 0, 0, 0
    
    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                if target_text in span['text']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = x0, y0, x1, y1

    return [target_x0, target_y0, target_x1, target_y1]

In [15]:
''' 
Code for  data extraction of Single File

'''

import fitz
import pandas as pd

file_name = "1040_file_180.pdf"
pdf_dir = f"./Data Extraction/Train Files/{file_name}"  # Replace with your PDF file path
doc = fitz.open(pdf_dir)
page = doc[0]

blocks = page.get_text("dict")['blocks']
text_blocks = [block for block in blocks if block['type'] == 0]


df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])

for data in datas:
    target_text = None
    for block in text_blocks:
        for line in block['lines']:
            for span in line['spans']:
                x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                        target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                    target_text = span['text']

    # Print the text at the given coordinates
    if target_text is not None:
        print(f"Text at coordinates ({target_x0}, {target_y0}, {target_x1}, {target_y1}): {target_text}")

        coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

        new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                   'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
        df.loc[len(df)] = new_row 
        
    else:
        print("")
df


Text at coordinates (100, 325, 650, 365): Richard
Text at coordinates (675, 325, 1300, 365): Pollard
Text at coordinates (1400, 1015, 1600, 1049): 92299.43
Text at coordinates (1400, 1049, 1600, 1083): 61947.0

Text at coordinates (1400, 1112, 1600, 1150): 62396.35
Text at coordinates (1400, 1150, 1600, 1184): 21766.83
Text at coordinates (1400, 1180, 1600, 1219): 1.0


Text at coordinates (1400, 1284, 1600, 1352): 62.0
Text at coordinates (1400, 1352, 1600, 1384): 8392.0
Text at coordinates (1400, 1384, 1600, 1417): 0.0
Text at coordinates (1400, 1417, 1600, 1450): 43300.28


Text at coordinates (1400, 1515, 1600, 1585): 96.0
Text at coordinates (1400, 1585, 1600, 1618): 9101.0
Text at coordinates (1400, 1618, 1600, 1650): 47258.18
Text at coordinates (1400, 1650, 1600, 1685): 84841.71

Text at coordinates (1400, 1715, 1600, 1752): 237.0
Text at coordinates (1400, 1748, 1600, 1785): 97554.13




,file_name,label,value,x0,y0,x2,y2
0,1040_file_180.pdf,Basic Information___First Name,Richard,116.899999,329.099994,203.025013,363.449987
1,1040_file_180.pdf,Basic Information___Last Name,Pollard,678.300010,325.900057,756.099955,360.250049
2,1040_file_180.pdf,Income___One_A,92299.43,1459.902785,1017.700026,1540.986633,1044.416640
3,1040_file_180.pdf,Income___One_B,61947.0,1467.658318,1052.900018,1537.930976,1079.616631
4,1040_file_180.pdf,Income___One_D,62396.35,1459.902785,1116.800012,1540.986633,1143.516625
5,1040_file_180.pdf,Income___One_E,21766.83,1460.652839,1151.900058,1541.736688,1178.616672
6,1040_file_180.pdf,Income___One_F,1.0,1472.263930,1586.499871,1531.725396,1613.216654
7,1040_file_180.pdf,Income___One_Z,62.0,1483.074951,1310.100047,1520.914035,1336.816661
8,1040_file_180.pdf,Income___Two_B,8392.0,1471.463860,1353.200022,1530.925327,1379.916636
9,1040_file_180.pdf,Income___Three_B,0.0,1488.480632,1386.800045,1515.508525,1413.516659


In [16]:
'''
Data extraction of multiple file present in a test dataset. 
'''
import fitz
import pandas as pd
df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])
i=1
while True:
    if i == 201:
        break
    file_name = f"1040_file_{i}.pdf"
    pdf_dir = f"./Data Extraction/Test Files/{file_name}"  
    try:
        doc = fitz.open(pdf_dir)
        page = doc[0]
    except :
        #if the file is not present in the directory, skip the file and continue
        print(f"File not found: {file_name}. Skipping...")
        i += 1
        continue

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    #Loop through the co ordinates present in a datas to find if the data is present in the bbox. 
    for data in datas:
        target_text = None
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                    if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                            target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                        target_text = span['text']

        # Add that co-ordinates in the data frame if the text is present in given  bbox
        if target_text is not None:
            

            coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

            new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                    'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
            df.loc[len(df)] = new_row 
        else:
            print("")
        
    i=i+1

    


pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# Save the extracted dataset into csv file
df.to_csv("./test.csv")
df

File not found: 1040_file_1.pdf. Skipping...
File not found: 1040_file_2.pdf. Skipping...
File not found: 1040_file_3.pdf. Skipping...
File not found: 1040_file_4.pdf. Skipping...
File not found: 1040_file_5.pdf. Skipping...
File not found: 1040_file_6.pdf. Skipping...
File not found: 1040_file_7.pdf. Skipping...
File not found: 1040_file_8.pdf. Skipping...
File not found: 1040_file_9.pdf. Skipping...













File not found: 1040_file_11.pdf. Skipping...
File not found: 1040_file_12.pdf. Skipping...
File not found: 1040_file_13.pdf. Skipping...
File not found: 1040_file_14.pdf. Skipping...
File not found: 1040_file_15.pdf. Skipping...





















File not found: 1040_file_18.pdf. Skipping...










File not found: 1040_file_20.pdf. Skipping...
File not found: 1040_file_21.pdf. Skipping...
File not found: 1040_file_22.pdf. Skipping...
File not found: 1040_file_23.pdf. Skipping...
File not found: 1040_file_24.pdf. Skipping...
File not found: 1040_file_25.pdf. Skipping...

,file_name,label,value,x0,y0,x2,y2
0,1040_file_10.pdf,Basic Information___First Name,Derrick,116.899999,329.099994,196.050008,363.449987
1,1040_file_10.pdf,Basic Information___Last Name,Wilson,678.300010,325.900057,753.299967,360.250049
2,1040_file_10.pdf,Income___One_B,30.0,1483.875020,1052.900018,1521.714105,1079.616631
3,1040_file_10.pdf,Income___One_D,49.0,1481.524997,1116.800012,1519.364081,1143.516625
4,1040_file_10.pdf,Income___One_F,60497.0,1466.858249,1183.900028,1537.130907,1210.616642
...,...,...,...,...,...,...,...
517,1040_file_187.pdf,Income___Six_B,14562.301,1455.247158,1487.399970,1547.142198,1514.116753
518,1040_file_187.pdf,Income___Seven,46053.4,1467.658318,1548.099942,1537.930976,1574.816725
519,1040_file_187.pdf,Income___Eight,49.0,1483.074951,1586.499871,1520.914035,1613.216654
520,1040_file_187.pdf,Income___Twelve,337.0,1478.469510,1717.499966,1527.119785,1744.216749


In [17]:
'''
Data extraction of multiple file present in a train dataset.
'''

import fitz
import pandas as pd
df = pd.DataFrame(columns=['file_name', 'label', 'value', 'x0', 'y0', 'x2', 'y2'])
i=1
while True:
    if i == 201:
        break
    file_name = f"1040_file_{i}.pdf"
    pdf_dir = f"./Data Extraction/Train Files/{file_name}"  
    try:
        doc = fitz.open(pdf_dir)
        page = doc[0]
    except :
        print(f"File not found: {file_name}. Skipping...")
        i += 1
        continue

    blocks = page.get_text("dict")['blocks']
    text_blocks = [block for block in blocks if block['type'] == 0]

    
       #Loop through the co ordinates present in a datas to find if the data is present in the bbox. 
    
    for data in datas:
        target_text = None
        for block in text_blocks:
            for line in block['lines']:
                for span in line['spans']:
                    x0, y0, x1, y1 = [coord * TARGET_DPI / scaleX for coord in span['bbox']]
                    target_x0, target_y0, target_x1, target_y1 = data["coordinate"]
                    if target_x0 <= x0 <= target_x1 and target_y0 <= y0 <= target_y1 and \
                            target_x0 <= x1 <= target_x1 and target_y0 <= y1 <= target_y1:
                        target_text = span['text']

        # Print the text at the given coordinates
        if target_text is not None:
            # print(f"Text at coordinates ({target_x0}, {target_y0}, {target_x1}, {target_y1}): {target_text}")

            coordinate = extract_text_from_coordinates(pdf_dir=pdf_dir, target_text=target_text)

            new_row = {"file_name": file_name, 'label': data["title"], "value": target_text,
                    'x0': coordinate[0], 'y0': coordinate[1], 'x2': coordinate[2], "y2": coordinate[3]}
            df.loc[len(df)] = new_row 
            # print(new_row)
            # print(coordinate)
        else:
            print("")
        
    i=i+1

    


pd.set_option('display.max_rows', 200)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
df.to_csv("./train.csv")
df









































































































File not found: 1040_file_10.pdf. Skipping...





















































File not found: 1040_file_16.pdf. Skipping...
File not found: 1040_file_17.pdf. Skipping...













File not found: 1040_file_19.pdf. Skipping...





























































































































File not found: 1040_file_31.pdf. Skipping...












































































































































File not found: 1040_file_46.pdf. Skipping...



































































































File not found: 1040_file_56.pdf. Skipping...
File not found: 1040_file_57.pdf. Skipping...





























File not found: 1040_file_61.pdf. Skipping...

























,file_name,label,value,x0,y0,x2,y2
0,1040_file_1.pdf,Basic Information___First Name,Maureen,116.899999,329.099994,215.549999,363.449987
1,1040_file_1.pdf,Basic Information___Last Name,Hoffman,678.300010,325.900057,772.775099,360.250049
2,1040_file_1.pdf,Income___One_F,23287.39,1461.452738,1183.900028,1542.536587,1210.616642
3,1040_file_1.pdf,Income___Three_B,8.0,1488.480632,1386.800045,1515.508525,1413.516659
4,1040_file_1.pdf,Income___Four_B,6.0,1479.269409,1450.699870,1527.919685,1477.416653
...,...,...,...,...,...,...,...
2182,1040_file_200.pdf,Income___Six_B,81325.78,1460.652839,1487.399970,1541.736688,1514.116753
2183,1040_file_200.pdf,Income___Nine,73661.413,1454.497274,1618.399896,1546.392314,1645.116679
2184,1040_file_200.pdf,Income___Ten,41023.34,1463.052707,1651.999919,1544.136556,1678.716702
2185,1040_file_200.pdf,Income___Eleven,75223.22,1462.252808,1683.999973,1543.336656,1710.716756


In [18]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Load the two CSV files into DataFrames
df_true = pd.read_csv("./Data Extraction/DE-TRAIN.csv")
df_pred = pd.read_csv("train.csv")



# Identify missing rows in file1 compared to file2
missing_rows = df_true[~df_true.isin(df_pred)].dropna()

# Display the missing rows
print("Missing Rows in file1 compared to file2:")
print(missing_rows)

Missing Rows in file1 compared to file2:
Empty DataFrame
Columns: [file_name, label, value, x0, y0, x2, y2]
Index: []


In [19]:

# Get the common column names
common_columns={'x0','y0','x2','y2'}

# Initialize a dictionary to store Mean Absolute Errors for each column
mae_dict = {}
# Initialize an empty DataFrame with the desired columns
accuracy_df = pd.DataFrame(columns=[f"{common_column}_error" for common_column in common_columns])

for common_column in common_columns:
    accuracy_df[f"{common_column}error"] = abs((df_true[f"{common_column}"] - df_pred[f"{common_column}"]) / df_true[f"{common_column}"])
    # Print the resulting DataFrame with accuracy for each row

for common_column in common_columns:
    mean_value = accuracy_df[f"{common_column}error"].mean()
    accuracy=1-mean_value
    print(f"Accuracy of {common_column}  ={accuracy*100}%")

Accuracy of y0  =99.4398760798181%
Accuracy of y2  =99.28303780212856%
Accuracy of x2  =88.71367570293397%
Accuracy of x0  =95.6973521223719%


In [20]:
# Specify the field for comparison (replace 'your_field' with the actual field name)
field_to_compare = ['value','label']

# Create a new column indicating whether the values are equal
for field in field_to_compare:
    df_pred['equal_values'] = (df_pred[f"{field}"] == df_true[f"{field}"]).astype(int)
    # Calculate accuracy
    accuracy = df_pred['equal_values'].mean() * 100
    # Display accuracy
    print(f"Accuracy of field{field}: {accuracy:.2f}%")

Accuracy of fieldvalue: 58.34%
Accuracy of fieldlabel: 100.00%
